## feature reduction

Data can be loaded downloaded [here](https://drive.google.com/drive/folders/1yZI5v3ws3b8GZMl_ACe4TO_qebdS2fUz?usp=sharing). The data is contained in the `srp_raw01.zip` and has to be moved to `/data/raw`.
The resulting folder structure looks like this:
`/data/raw/n`, `/data/raw/o` and `/data/raw/x`.

In this notebook the feature space of the data is reduced.

By reducing redundancy in data the model decreases in size and may improve in performace, because less weights and biases have to be adjusted.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.utils import reset_and_distribute_data

reset_and_distribute_data(raw, processed, [400,0,100])

In essence the only change undertaken in this notebook is the `color_mode` parameter given to the `flow_from_directory` function.

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


The `model.summary` shows that the number of parameters is almost divided by a factor of three (was 99491 in previous models). This is translating directly into the size of the saved model.

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 32, 1)))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                32800     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 33,955
Trainable params: 33,955
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.optimizers import SGD

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', 'srp62', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
mkdir(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
20/20 [==============================] - 2s 85ms/step - loss: 1.1105 - acc: 0.3475
Epoch 2/20
20/20 [==============================] - 1s 75ms/step - loss: 1.0981 - acc: 0.4050
Epoch 3/20
20/20 [==============================] - 2s 77ms/step - loss: 1.0781 - acc: 0.4750
Epoch 4/20
20/20 [==============================] - 1s 66ms/step - loss: 1.0374 - acc: 0.4500
Epoch 5/20
20/20 [==============================] - 1s 66ms/step - loss: 1.0325 - acc: 0.4775
Epoch 6/20
20/20 [==============================] - 1s 62ms/step - loss: 0.9900 - acc: 0.5675
Epoch 7/20
20/20 [==============================] - 1s 68ms/step - loss: 0.9358 - acc: 0.6150
Epoch 8/20
20/20 [==============================] - 1s 66ms/step - loss: 0.8945 - acc: 0.6375
Epoch 9/20
20/20 [==============================] - 1s 63ms/step - loss: 0.8521 - acc: 0.7125
Epoch 10/20
20/20 [==============================] - 1s 69ms

In [6]:
model.evaluate_generator(test_generator)

[0.6483220954736074, 0.7866667]

The resulting model has 295kb, whereas the old models were 807kb big. The model is much smaller now without loosing any measurable data.

In [7]:
model_path = join('models', 'devel', 'srp62.h5')
model.save(model_path)